In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F

In [2]:
block_size = 512
nembed = 64
nembed2 = 64

batch_size = 64
device = 'cuda'

In [3]:
text = open("input.txt", 'r').read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
encode = lambda x: [stoi[s] for s in x]
decode = lambda x: "".join([itos[i] for i in x])

data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size, ))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [261]:
b, t, t = 2, 5, 5
f = torch.randn(b, t)
i = torch.randn(b, t)

In [262]:
f.sigmoid()

tensor([[0.2073, 0.6400, 0.5589, 0.7215, 0.1355],
        [0.4899, 0.4087, 0.6986, 0.7709, 0.4859]])

In [272]:
%%time
F = torch.zeros(b, t, t)
for l in range(t):
    for j in range(t):
        if j == l:
            F[:, l, j] = torch.ones_like(F[:, l, j])
        if l > j:
            F[:, l, j] = f[:, j + 1: l + 1].sigmoid().prod(1)

I = torch.zeros(b, t, t)
for l in range(t):
    for j in range(t):
        if l >= j:
            I[:, l, j] = i[:, j]

CPU times: total: 0 ns
Wall time: 1 ms


In [276]:
F 

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7311, 1.0000, 0.0000, 0.0000, 0.0000],
         [0.5344, 0.7311, 1.0000, 0.0000, 0.0000],
         [0.3907, 0.5344, 0.7311, 1.0000, 0.0000],
         [0.2856, 0.3907, 0.5344, 0.7311, 1.0000]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7311, 1.0000, 0.0000, 0.0000, 0.0000],
         [0.5344, 0.7311, 1.0000, 0.0000, 0.0000],
         [0.3907, 0.5344, 0.7311, 1.0000, 0.0000],
         [0.2856, 0.3907, 0.5344, 0.7311, 1.0000]]])

In [274]:
f = torch.nn.functional.pad(f[:,1:], (0, 1,), value = 1)
F2 = (f[:, None, :].expand(-1, t, -1).sigmoid().tril(-1)  
      + torch.ones(t, t)[None, :].triu()).flip(-1).cumprod(-1).flip(-1)
F2.tril()

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7311, 1.0000, 0.0000, 0.0000, 0.0000],
         [0.5344, 0.7311, 1.0000, 0.0000, 0.0000],
         [0.3907, 0.5344, 0.7311, 1.0000, 0.0000],
         [0.2856, 0.3907, 0.5344, 0.7311, 1.0000]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7311, 1.0000, 0.0000, 0.0000, 0.0000],
         [0.5344, 0.7311, 1.0000, 0.0000, 0.0000],
         [0.3907, 0.5344, 0.7311, 1.0000, 0.0000],
         [0.2856, 0.3907, 0.5344, 0.7311, 1.0000]]])

In [4]:
class mLSTMCell(nn.Module):
    def __init__(self):
        super().__init__()
        self.qkv= nn.Linear(nembed, nembed2 * 3)
        self.fi = nn.Linear(nembed, 1 * 2)
        self.o = nn.Linear(nembed, nembed2)
    
    def rnn_forward(self, xt, nprev, cprev):
        q, k, v, = self.qkv(xt).split(nembed2, dim = -1) # b, t, c
        i, f = self.fi(xt).split(1, dim = -1)
        o = self.o(xt)
        k = k * (nembed2 ** -0.5)
        
        f = torch.sigmoid(f)
        i = torch.exp(i)
        o = torch.sigmoid(o)
        #cprev = b, c, c
        #nprev = b, t, c

        ct = f[..., None] * cprev + i[..., None] * (v[..., None] * k[..., None, :])
        nt = f * nprev + i * k
        ht_bar = ct @ q[..., None] / torch.abs(nt[..., None, :] @ q[..., None]).clamp(min = 1)
        ht = o * ht_bar[..., -1]
        return ht, nt, ct

    def parallel_forward(self, xt):
        b, t, c = xt.size()
        q, k, v, = self.qkv(xt).split(nembed2, dim = -1) # b, t, c
        i, f= self.fi(xt).split(1, dim = -1)
        f = f.squeeze(-1)
        i = i.squeeze(-1)
        o = self.o(xt)
        k = k * (nembed2 ** -0.5)

        # F = torch.zeros(b, t, t).to(q.device)
        # for l in range(t):
        #     for j in range(t):
        #         if j == l:
        #             F[:, l, j] = torch.ones_like(F[:, l, j])
        #         if l > j:
        #             F[:, l, j] = f[:, j + 1: l + 1].sigmoid().prod(1)
        f = torch.nn.functional.pad(f[:,1:], (0, 1,), value = 1)
        F = (f[:, None, :].expand(-1, t, -1).sigmoid().tril(-1)  
             + torch.ones(t, t)[None, :].triu().to(q.device)).flip(-1).cumprod(-1).flip(-1)
        F = F.tril()
        # I = torch.zeros(b, t, t).to(q.device)
        # for l in range(t):
        #     for j in range(t):
        #         if l >= j:
        #             I[:, l, j] = i[:, j]

        I = torch.zeros(b, t, t).to(q.device)
        I = torch.tril(I + i[:, None, :])
        # print(f,'\n',F, '\n',I)
        Cbar = (q @ k.transpose(-2 , -1)) * F * I.exp()
        C = Cbar / torch.abs(Cbar.sum(2, keepdim = True)).clamp(min = 1)
        hbar = C @ v
        return hbar * o.sigmoid()


In [6]:
xt = torch.randn(2, 5, 64)
m = mLSTMCell()

h = []
cprev = torch.zeros(2, 1, 64, 64)
nprev = torch.zeros(2, 1, 64)
for t in range(5):
    ht, nprev, cprev = m.rnn_forward(xt[:, t: t + 1], nprev, cprev)
    h.append(ht)
p1 = torch.cat(h, dim = 1)

p2 = m.parallel_forward(xt)

p2[0, -2], p1[0, -2]

(tensor([ 0.0589,  0.0974,  0.2051, -0.0566,  0.0165, -0.0482,  0.1165,  0.0740,
          0.0615,  0.1346, -0.0803, -0.0192, -0.1159,  0.2436, -0.1808,  0.0362,
          0.0420, -0.0451,  0.0059, -0.0804, -0.1492, -0.2376, -0.0786, -0.1149,
          0.0568, -0.0988,  0.0135, -0.0613,  0.0287,  0.1159,  0.0419, -0.1804,
         -0.1795, -0.1460, -0.0021,  0.1443, -0.0487, -0.0275, -0.1300,  0.0016,
          0.0272,  0.0595,  0.0927,  0.0931,  0.0313, -0.0498, -0.0746,  0.0177,
          0.0545,  0.0658,  0.0346,  0.0038,  0.1204,  0.1313, -0.0329,  0.0241,
          0.0091,  0.0742,  0.0119, -0.0508,  0.1329,  0.0128,  0.1369, -0.0510],
        grad_fn=<SelectBackward0>),
 tensor([ 0.0589,  0.0974,  0.2051, -0.0566,  0.0165, -0.0482,  0.1165,  0.0740,
          0.0615,  0.1346, -0.0803, -0.0192, -0.1159,  0.2436, -0.1808,  0.0362,
          0.0420, -0.0451,  0.0059, -0.0804, -0.1492, -0.2376, -0.0786, -0.1149,
          0.0568, -0.0988,  0.0135, -0.0613,  0.0287,  0.1159,  0.0419, 

In [7]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.start = nn.Parameter(torch.zeros(1, nembed2))
        self.cstart = nn.Parameter(torch.zeros(1, nembed2, nembed2))
        self.emb = nn.Embedding(vocab_size, nembed)
        self.cell = mLSTMCell()
        self.proj = nn.Linear(nembed2, vocab_size)
    
    def forward(self, idx, targets = None, tp = 0):
        B, T = idx.size()
        emb = self.emb(idx)
        #sequentially iterate over inputs and update torche rnn state each tick
        if tp:
            nprev = self.start.expand((B, -1, -1))
            cprev = self.cstart.expand((B, -1, -1, -1))
            hiddens = []
            for i in range(T):
                xt = emb[:, i: i + 1, :]
                # xt = nn.LayerNorm(nembed).cuda()(xt)
                ht, nt, ct = self.cell.rnn_forward(xt, nprev, cprev)
                nprev, cprev = nt, ct
                hiddens.append(ht)
            hiddens = torch.cat(hiddens, 1)
        else:
            hiddens = self.cell.parallel_forward(emb)
        logits = self.proj(hiddens)

        loss = None
        if targets is not None:
            loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

In [8]:
model = RNN().to(device)
print("Model Parameters: ", sum(p.numel() for p in model.parameters()))

Model Parameters:  29315


In [9]:
max_iters = 3000
eval_iters = 50
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

from tqdm import tqdm
pbar = tqdm(range(max_iters))

for j in pbar:
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb, tp = 0)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none = True)
    pbar.set_description(f"Loss {loss.item():.4f}")
    # if j % 50== 0:
    #     losses = estimate_loss()
    #     print(f"\nStep {j}: train_loss {losses['train']:.4f}  val_loss {losses['val']:.4f}")

Loss 2.0031:  29%|██▉       | 874/3000 [01:08<02:47, 12.68it/s]


KeyboardInterrupt: 

In [10]:
model(xb)[0][2][6], model(xb, tp = 1)[0][2][6]

(tensor([ 9.3108,  2.3435, -3.4952, -3.8625, -3.2206,  2.1256, -0.8196, -0.3650,
         -1.7328,  1.2867, -1.0852, -3.1300, -1.4932,  5.9854,  5.6188,  5.6602,
          4.8232,  3.5847,  5.1075,  5.1896,  5.6720,  5.9690,  2.7234,  4.1191,
          4.9957,  5.5276,  4.8702,  4.7934,  4.3832,  3.1226,  3.3231,  5.3991,
          6.6216,  2.5352,  2.8224,  6.4438, -3.6266,  4.2546, -4.9650, -1.6632,
         -1.5166, -3.8289, -4.6490, -3.1766, -4.1552, -2.8709, -3.3555, -1.9243,
         -2.7107, -4.1332, -1.3281, -3.1763, -3.2467, -3.7656, -0.8999, -4.5163,
         -0.4250, -2.0757, -2.4817, -0.4103, -3.0978,  1.1716, -3.5617, -2.8300,
         -4.9416], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([ 9.3108,  2.3435, -3.4952, -3.8625, -3.2207,  2.1256, -0.8196, -0.3650,
         -1.7328,  1.2867, -1.0852, -3.1300, -1.4932,  5.9854,  5.6188,  5.6602,
          4.8232,  3.5847,  5.1075,  5.1896,  5.6720,  5.9690,  2.7234,  4.1192,
          4.9957,  5.5277,  4.8703,  4.7934, 

In [26]:
@torch.no_grad()
def generate(model, idx, steps):
    for _ in range(steps):
        idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
        logits, _ = model(idx_cond, tp = 0)
        logits = logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx


In [27]:
print(decode((generate(model, torch.tensor([[0]], dtype = torch.long, device= 'cuda'), 1000))[0].tolist()))


Sour me fall mings be imener; ther's word thou comeses here his sous lash; love cren ounge and cliking kneys
thou a alll sup lows
Stall:
Nod Ureces.

QUEENCENVIG:
Cown, prand he that Poucusterste;
In dour truch theme, agrt for pocl'd me
And ea thimbe? hoI earng.

Ulore, onitelles mor ecent sily.

DUKE TELOYBUENTCURY:
The gones. BUre do?

Goyst frole ben pirst cout mentwak forsiot thimente son, your me
RICHARETIO:
Comas dothizetrier's dite.
Nlie one wopld pree whold.

What me lock? EN:
Buntll out your the din alld, cant gradthse a sunot buate.
Wit theesesse,
As oulde Whold for with sp ath the I not un de;
WARYour depsuch
LWINA:
We!
LONES Clay urcus tis of oftis fart them isell. ifes mose: thany silme:
That come,
Or rence ast quado? Th hyou I poode w with nechtid
ith peady, Peed rown.

Thonis the tword wild hus-deestat: for do ucas, breold,
'lobdour the item;
Thiria HEST:
He thy willt I fil'd wast for Mit, itl guis yourfl
For Arand page
KE VI:
He formm foolat, by armm beme of a for but 